In [ ]:
import os
from IPython.display import HTML, display

class Benchmark:
    def __init__(self, name):
        self.name = name
        self.field = []
        self.results = []
    
    def load(self, path, cases=['bench', 'regprof3']):
        self.field = cases
        for case in cases:
            result = {}
            files = [path+'/'+f for f in os.listdir(path) 
                        if os.path.isfile(path+'/'+f)]
            for file in files:
                if file.endswith('.'+case):
                    with open(file, 'r') as f:
                        _ , name = os.path.split(file)
                        result[name] = f.readlines()
            self.results.append(result)
        return len(self.results) != 0
    
    def __str__(self) -> str:
        s = ""
        for i in range(len(self.field)):
            s += self.field[i]
            for j in self.results[i].keys():
                s += ("\n    " + j)
            s += '\n'
        return s

def load_build_folder(build_folder = '../build/benchmarks/'):
    """Load a build folder and return a list of results"""
    benchmarks = []
    build_folder = os.path.abspath(build_folder)
    print(build_folder)
    folders = [f for f in os.listdir(build_folder) 
                        if os.path.isdir(f)]    
    print(folders)
    for folder in folders:
        B = Benchmark(folder)
        if B.load(build_folder+'/'+folder):
            benchmarks.append(B)
    return benchmarks

def printTable(data, header=None):
    def gen_row(row, el='td'):
        return '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row))
    display(HTML('<table><tr>{}</tr></table>'.format(
        (gen_row(header, 'th') + '</tr><tr>' if header else '') + 
        '</tr><tr>'.join(
            gen_row(row) for row in data)
        )
    ))

Benchmarks = load_build_folder()

First, let's show push and pop counts